# Producing solutions

In [1]:
import numpy as np
from numpy import array

from Functions import D_X,D_Y,D,div,evolve,forward_euler, crank_nicholson, backwards_euler, alpha, dp, M, dE, E, dw, alt_evolve,alt_forward_euler

from Parameters import h,N,X,Y,Omega, D1,D2,rho_10,rho_20, V_1,V_2,DV_1,DV_2, savefile

## Evolving the system
---


In [2]:
rho_t_1,rho_t_2,t, data1 = evolve(.1*h**2,1,0.0005,'forward_euler')   # this is the expanded scheme. Arguments are dt, final time, minimum_value of ||dp/dt||_{L^2}, ODE solver method

1% completed. Current value of dp/dt = 7.014932328777084
2% completed. Current value of dp/dt = 5.600897075787989
3% completed. Current value of dp/dt = 4.780150848140232
4% completed. Current value of dp/dt = 4.053264108468363
5% completed. Current value of dp/dt = 3.4050821710666264
6% completed. Current value of dp/dt = 2.8330681492963405
7% completed. Current value of dp/dt = 2.334371533142627
8% completed. Current value of dp/dt = 1.905564196167054
9% completed. Current value of dp/dt = 1.5421289037667372
10% completed. Current value of dp/dt = 1.238377182103955
11% completed. Current value of dp/dt = 0.987758932063952
12% completed. Current value of dp/dt = 0.7833337071466815
13% completed. Current value of dp/dt = 0.6182281432360814
14% completed. Current value of dp/dt = 0.4859889414452777
15% completed. Current value of dp/dt = 0.3808057798457767
16% completed. Current value of dp/dt = 0.29761658507984
17% completed. Current value of dp/dt = 0.23212374639439018
18% completed. 

In [ ]:
rho_t_1_alt,rho_t_2_alt,t_alt, data2 = alt_evolve(.1*h**2,1,0.0005,'forward_euler')  # this is the gradient flow scheme. Arguments are dt, final time, minimum_value of ||dp/dt||_{L^2}, ODE solver method

In [ ]:
title = 'data1'
np.save(savefile+'/'+title + '.npy', data1)
title = 'data2'
np.save(savefile+'/'+title + '.npy', data2)

In [ ]:
data1 = np.load(savefile+'/data1' + '.npy',allow_pickle='TRUE').item()
data2 = np.load(savefile+'/data2' + '.npy',allow_pickle='TRUE').item()

rho_t_1 = data1['rho_t_1']
rho_t_2 = data1['rho_t_2']
t = data1['t']

rho_t_1_alt = data2['rho_t_1']
rho_t_2_alt = data2['rho_t_2']
t_alt = data2['t']

## Animations
---
Here an animation is created showing the evolution of the individual densities along side the energy potentials $V$.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation

def makePlot(Time_length,Title,rho_t_1,rho_t_2,t):
    scale = int(rho_t_1.shape[0]/(Time_length*20))
    fig = plt.figure(figsize=(12,5))
    ax = plt.subplot(1,2,1)   
    txt_title = ax.set_title('')
    ax.set_xlim(( 0, 1))            
    ax.set_ylim((-1, 1))
    line1, = ax.plot([], [], 'b', lw=2, label = 'rho_1')  
    line2, = ax.plot([], [], 'r', lw=2, label = 'rho_2')  
    line3, = ax.plot([], [], 'g', lw=2, label = 'V_1')  
    line4, = ax.plot([], [], 'y', lw=2, label = 'V_2')  
    ax.legend()
    def animate(i): 
         # grab a random integer to be the next y-value in the animation
        x = np.linspace(0,1-h,N)
        y1 = rho_t_1[scale*i,0,:] #+  rho_t_2[i,:,0]
        y2 = rho_t_2[scale*i,0,:]
        line1.set_data(x, y1)
        line2.set_data(x, y2)
        line3.set_data(x, V_1(x,0))
        line4.set_data(x,  V_2(x,0))
        txt_title.set_text('t = {0:4f}'.format(t[scale*i]))
        return line1,line2,line3,line4,
    ani = FuncAnimation(fig, animate, frames=Time_length*20, interval=20,  blit=True)
    ani.save(savefile+'/'+Title + '.mp4', writer = 'ffmpeg', fps = 30)

def makePlotMatrixDeterminant(Time_length,Title,rho_t_1,rho_t_2,t):
    scale = int(rho_t_1.shape[0]/(Time_length*20))
    fig = plt.figure(figsize=(12,5))
    ax = plt.subplot(1,2,1)   
    txt_title = ax.set_title('')
    ax.set_xlim(( 0, 1))                
    Max_value = 1e-6
    for j in range(0,len(t)):
        Mob =M(rho_t_1[j,0,:],rho_t_2[j,0,:])
        x = np.linspace(0,1-h,N)
        y1 =Mob[0,0,:]*Mob[1,1,:] - Mob[0,1,:]*Mob[1,0,:]
        if np.amax(abs(y1)) > Max_value:
            Max_value = np.amax(abs(y1))
    
    ax.set_ylim((-1.5*Max_value,1.5*Max_value))    
    line1, = ax.plot([], [], 'b', lw=2, label = 'det(M)')
    ax.legend()
    def animate(i): 
         # grab a random integer to be the next y-value in the animation
        Mob =M(rho_t_1[scale*i,0,:],rho_t_2[scale*i,0,:])
        x = np.linspace(0,1-h,N)
        y1 =Mob[0,0,:]*Mob[1,1,:] - Mob[0,1,:]*Mob[1,0,:]
        line1.set_data(x, y1)
        txt_title.set_text('t = {0:4f}'.format(t[scale*i]))
        return line1,
    ani = FuncAnimation(fig, animate, frames=Time_length*20, interval=20,  blit=True)
    ani.save(savefile+'/'+Title + '.mp4', writer = 'ffmpeg', fps = 30)

In [ ]:
makePlot(20,'System evolution expanded method',rho_t_1,rho_t_2,t)  # Arguments are length of video, title to save file, data from computed solution.
#makePlot(20,'System evolution  gradient flow method',rho_t_1_alt,rho_t_2_alt,t_alt)

makePlotMatrixDeterminant(20,'Expanded method Matrix determinant',rho_t_1,rho_t_2,t) # This plots the determinant of the mobiliti matrix, same input structure as above
#makePlotMatrixDeterminant(20,'Gradient flow method  Matrix determinant',rho_t_1_alt,rho_t_2_alt,t_alt)

## Snapshots
---

In [ ]:
def makeSnapshots(title,rho_t_1,rho_t_2,t):
    counter = 0
    for j in (0,int(len(t)/2), len(t)-2):
        fig = plt.figure(figsize=(7,7))
        ax = plt.subplot(1,1,1)   
        txt_title = ax.set_title('t = {0:2f}'.format(t[j]))
        ax.set_xlim(( 0, 1))            
        ax.set_ylim((-1, 1))
        line1, = ax.plot([], [], 'b', lw=2, label = 'rho_1')  
        line2, = ax.plot([], [], 'r', lw=2, label = 'rho_2') 
        line3, = ax.plot([], [], 'g', lw=2, label = 'V_1') 
        line4, = ax.plot([], [], 'y', lw=2, label = 'V_2') 

        x = np.linspace(0,1-h,N)
        y1 = rho_t_1[j,0,:]
        line1.set_data(x, y1)
        y2 = rho_t_2[j,0,:]
        line2.set_data(x, y2)
        line3.set_data(x, V_1(x,0))
        line4.set_data(x,  V_2(x,0))
        ax.legend()
        plt.savefig(savefile+'/'+title + ' ' +str(counter) +'.jpg')
        counter +=1

In [ ]:
#makeSnapshots('expanded method stable (constant) values',rho_t_1,rho_t_2,t)
#makeSnapshots('Gradient method stable (constant) values',rho_t_1_alt,rho_t_2_alt,t_alt)

#makeSnapshots('expanded method stable values',rho_t_1,rho_t_2,t)
#makeSnapshots('Gradient method stable values',rho_t_1_alt,rho_t_2_alt,t_alt)

#makeSnapshots('expanded method unstable values',rho_t_1,rho_t_2,t)
#makeSnapshots('Gradient method unstable values',rho_t_1_alt,rho_t_2_alt,t_alt)

makeSnapshots('expanded method unstable (refined) values',rho_t_1,rho_t_2,t)  # Produces three images of evolution of system
#makeSnapshots('Gradient method unstable (refined) values',rho_t_1_alt,rho_t_2_alt,t_alt)

## Entropy
---

Plotting evolution of Entropy functional

In [ ]:
def Entropy_plot(title,rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt):
    Entropy =0*t
    Entropy_alt =0*t_alt

    for j in range(0,len(t)):
        Entropy[j] = E(rho_t_1[j],rho_t_2[j])
    for j in range(0,len(t_alt)):
        Entropy_alt[j] = E(rho_t_1_alt[j],rho_t_2_alt[j])
    fig = plt.figure(figsize=(12,5))
    plt.plot(t,Entropy, label = 'expanded scheme')
    plt.plot(t_alt,Entropy_alt, label = 'gradient flow scheme')
    plt.legend()
    plt.savefig(savefile+'/'+title+'.jpg')
    plt.show()
    

In [ ]:
#Entropy_plot('Stable (constant) initial entropy',rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt)

#Entropy_plot('Unstable initial entropy',rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt)

Entropy_plot('Unstable (refined) initial entropy',rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt) 

Plotting different time derivatives of the entropy functional.

In [ ]:
def dt_Entropy_plot(title,rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt):
    Entropy =0*t
    Entropy_alt =0*t_alt

    for j in range(0,len(t)):
        Entropy[j] = E(rho_t_1[j],rho_t_2[j])
    for j in range(0,len(t_alt)):
        Entropy_alt[j] = E(rho_t_1_alt[j],rho_t_2_alt[j])
    dt_entropy =0*t
    for j in range(0,len(t)):
        dt_entropy[j] = dE(rho_t_1[j],rho_t_2[j])
    dt_entropy_alt =0*t_alt
    for j in range(0,len(t_alt)):
        dt_entropy_alt[j] = dE(rho_t_1_alt[j],rho_t_2_alt[j])

    fig = plt.figure(figsize=(12,5))
    plt.plot(t,dt_entropy,label = 'PDE time derivative, expanded scheme')
    plt.plot(t[:-1],(Entropy[1:]-Entropy[:-1])/(t[1]-t[0]), label = 'Numerical time derivative, expanded scheme')
    plt.plot(t_alt,dt_entropy_alt,label = 'PDE time derivative, gradient flow scheme')
    plt.plot(t_alt[:-1],(Entropy_alt[1:]-Entropy_alt[:-1])/(t_alt[1]-t_alt[0]), label = 'Numerical time derivative, gradient flow scheme')
    plt.legend()
    plt.savefig(savefile+'/'+title+'.jpg')
    plt.show()
    return dt_entropy

In [ ]:
#dt_entropy= dt_Entropy_plot('Stable (constant) initial time derivative entropy',rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt)

#dt_entropy= dt_Entropy_plot('Unstable initial time derivative entropy',rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt)

dt_entropy= dt_Entropy_plot('Unstable (refined) initial time derivative entropy',rho_t_1,rho_t_2,t,rho_t_1_alt,rho_t_2_alt,t_alt)

Comparing forward-euler with Crank-Nicholson. It was found that both produced solutions that were reasonably close, however the forward-euler method runs significantly quicker. Conclusion: for most experiments it is sufficient to use the forward_euler method.

In [ ]:
import time

start_time = time.time()

rho_t_1_forward_euler,rho_t_2_forward_euler,t_forward_euler =  evolve(.1*h**2,1,0.0005,'forward_euler')
forward_euler_time = time.time() - start_time
rho_t_1_crank_nicholson,rho_t_2_crank_nicholson,t_crank_nicholson = evolve(.1*h**2,1,0.0005,'crank_nicholson')  
crank_nicholson_time = time.time() - start_time - forward_euler_time

error = np.sqrt(np.sum((rho_t_1_forward_euler - rho_t_1_crank_nicholson)**2 +(rho_t_2_forward_euler- rho_t_2_crank_nicholson)**2)*h**2)/(t_forward_euler[-1] - t_forward_euler[0])

print('error =' + str(error))
print('forward_euler took ' + str(forward_euler_time) + ' seconds')
print('crank_nicholson took ' + str(crank_nicholson_time) + ' seconds')